<a href="https://colab.research.google.com/github/SnitikaDash/SHL_GenAI_AssessmentRecommendationSystem/blob/main/SHLRSCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium webdriver-manager beautifulsoup4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 30.2 MB/s eta 0:00:00


In [ ]:
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,804 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,683 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,783

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import re


In [ ]:
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Set path for ChromeDriver (installed in previous step)
driver = webdriver.Chrome(options=chrome_options)


In [ ]:
def extract_metadata_from_text(page_text):
    metadata = {
        "Duration": "Not found",
        "Test Type": "Not found",
        "Remote Testing Support": "Not found",
        "Adaptive/IRT Support": "Not found"
    }

    # Duration (new pattern)
    duration_match = re.search(r'Completion Time.*?=\s*(\d+)', page_text, re.IGNORECASE)
    if duration_match:
        metadata["Duration"] = f"{duration_match.group(1)} minutes"

    # Test Type (look for single letter codes like B, P, R etc.)
    test_type_match = re.search(r'Test Type:\s*([A-Z](?:\s*[A-Z])*)', page_text)
    if test_type_match:
        metadata["Test Type"] = test_type_match.group(1).strip()

    # Remote Testing (try variations)
    remote_testing_match = re.search(r'Remote Testing:\s*(Yes|No|Available|Not available)', page_text, re.IGNORECASE)
    if remote_testing_match:
        metadata["Remote Testing Support"] = remote_testing_match.group(1).strip()

    # Adaptive/IRT Support
    if re.search(r'\b(adaptive|IRT|Item Response Theory)\b', page_text, re.IGNORECASE):
        metadata["Adaptive/IRT Support"] = "Yes"

    return metadata


In [ ]:
data = []
product_urls = ["https://www.shl.com/en/assessments/"]  # Example URL list

for url in product_urls:
    driver.get(url)
    time.sleep(5)  # Allow time for the page to load

    try:
        text = driver.find_element(By.TAG_NAME, "body").text
        print(f"✅ Loaded: {url}")
    except:
        print(f"❌ Failed to load: {url}")
        text = ""

    # Extract metadata
    metadata = extract_metadata_from_text(text)
    data.append({
        'URL': url,
        'Duration': metadata["Duration"],
        'Test Type': metadata["Test Type"],
        'Remote Testing Support': metadata["Remote Testing Support"],
        'Adaptive/IRT Support': metadata["Adaptive/IRT Support"]
    })


✅ Loaded: https://www.shl.com/en/assessments/


In [ ]:
url = "https://www.shl.com/en/assessments/"
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')


In [ ]:
# Example: Extract all assessment titles
titles = [tag.get_text(strip=True) for tag in soup.select("h2, h3")]
print(titles[:10])

# Extract assessment cards or sections
sections = soup.find_all('section')  # Or use class-based selectors
for sec in sections:
    heading = sec.find(['h2', 'h3'])
    para = sec.find('p')
    if heading and para:
        print("Title:", heading.get_text(strip=True))
        print("Description:", para.get_text(strip=True))
        print("---")


['Outdated browser detected', 'Make Wise Talent Decisions', 'Drive Hiring Efficiency', 'Delight Your Candidates', '', 'Our Unmatched Portfolio of Assessments', 'Behavioral Assessments', 'Virtual Assessment & Development Centers', 'Personality Assessments', 'Cognitive Assessments']


In [ ]:
data = []

# Assuming sections contains the relevant data
sections = soup.find_all('section')
for sec in sections:
    heading = sec.find(['h2', 'h3'])
    para = sec.find('p')
    if heading and para:
        title = heading.get_text(strip=True)
        description = para.get_text(strip=True)
        data.append({
            "title": title,
            "description": description
        })

# Convert to DataFrame
df = pd.DataFrame(data)
df.to_csv("shl_assessments.csv", index=False)
print("Saved:", df.shape)


Saved: (0, 0)


In [ ]:
# Check the columns in the DataFrame
print(df.columns)

# Check the first few rows of the DataFrame
print(df.head())


RangeIndex(start=0, stop=0, step=1)
Empty DataFrame
Columns: []
Index: []


In [ ]:
data = []

# Scrape all sections
sections = soup.find_all('section')
for sec in sections:
    heading = sec.find(['h2', 'h3'])
    para = sec.find('p')
    if heading and para:
        title = heading.get_text(strip=True)
        description = para.get_text(strip=True)
        data.append({
            "title": title,
            "description": description
        })

# Check the DataFrame after extraction
df = pd.DataFrame(data)

# Print column names and first few rows to verify correct extraction
print(df.columns)
print(df.head())



RangeIndex(start=0, stop=0, step=1)
Empty DataFrame
Columns: []
Index: []


In [ ]:
!apt update
!apt install -y chromium-browser chromium-chromedriver libnss3 libgdk-pixbuf2.0-0 libasound2


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
36 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL you want to scrape
url = "https://www.example.com"

# Send a GET request to the webpage
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the content of the webpage with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Example: Extract the title of the page
    title = soup.title.string
    print(f"Page title: {title}")


    for link in soup.find_all('a', href=True):
        print(link['href'])
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


Page title: Example Domain
https://www.iana.org/domains/example


In [ ]:
# Find all hyperlinks (a tags) on the page
for link in soup.find_all('a', href=True):
    print(link['href'])


https://www.iana.org/domains/example


In [ ]:
df = pd.DataFrame(data)

# Check if columns exist
if 'description' not in df.columns:
    print("Description column is missing!")

    df['description'] = ""
else:
    print("Description column found.")

# Print the DataFrame to inspect
print(df.head())


Description column is missing!
Empty DataFrame
Columns: [description]
Index: []


In [ ]:
# Scrape all sections
sections = soup.find_all('section')
for sec in sections:
    heading = sec.find(['h2', 'h3'])
    para = sec.find('p')

    # Debugging print statements
    print(f"Heading: {heading}")
    print(f"Paragraph: {para}")

    if heading and para:
        title = heading.get_text(strip=True)
        description = para.get_text(strip=True)
        data.append({
            "title": title,
            "description": description
        })

# Check the DataFrame after extraction
df = pd.DataFrame(data)

# Print column names and first few rows to verify correct extraction
print(df.columns)
print(df.head())


RangeIndex(start=0, stop=0, step=1)
Empty DataFrame
Columns: []
Index: []


In [ ]:
# Scrape all sections
sections = soup.find_all('section')
for sec in sections:
    heading = sec.find(['h2', 'h3'])
    para = sec.find('p')

    # Ensure para is defined and handle missing descriptions
    description = para.get_text(strip=True) if para else "No description available"

    # Debugging: Print section data
    print(f"Heading: {heading}")
    print(f"Description: {description}")

    if heading:  # Ensure heading exists before appending to data
        title = heading.get_text(strip=True)
        data.append({
            "title": title,
            "description": description
        })

# Check the DataFrame after extraction
df = pd.DataFrame(data)

# Print column names and first few rows to verify correct extraction
print(df.columns)
print(df.head())


RangeIndex(start=0, stop=0, step=1)
Empty DataFrame
Columns: []
Index: []


In [ ]:
# Check the data being collected
for entry in data[:5]:  # Display the first 5 entries to verify
    print(entry)


In [ ]:
df = pd.DataFrame(data)

# Check the DataFrame to verify columns
print(df.columns)
print(df.head())


RangeIndex(start=0, stop=0, step=1)
Empty DataFrame
Columns: []
Index: []


In [ ]:
data = []
# Scrape all sections
sections = soup.find_all('section')
for sec in sections:
    heading = sec.find(['h2', 'h3'])
    para = sec.find('p')
    if heading:
        title = heading.get_text(strip=True)
    else:
        title = "No title available"

    description = para.get_text(strip=True) if para else "No description available"

    data.append({
        "title": title,
        "description": description
    })


In [ ]:
df = pd.DataFrame(data)

# Verify column names and some sample data
print(df.columns)
print(df.head())


RangeIndex(start=0, stop=0, step=1)
Empty DataFrame
Columns: []
Index: []


In [ ]:
print(df.columns.tolist())


[]


In [ ]:
print(soup.prettify()[:1000])  # print the first 1000 chars


<!DOCTYPE html>
<html>
 <head>
  <title>
   Example Domain
  </title>
  <meta charset="utf-8"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <style type="text/css">
   body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
  </style>
 </head>
 <body>
  <div>
   <h1>
    Example Domain
   </h1>
   <p>
    This dom

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

options = Options()
options.add_argument("--headless")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)
driver.get("https://www.shl.com/en/assessments/")

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")


In [ ]:
data = []

headings = soup.find_all(['h2', 'h3'])
for heading in headings:
    para = heading.find_next_sibling('p')
    title = heading.get_text(strip=True)
    description = para.get_text(strip=True) if para else "No description available"
    data.append({"title": title, "description": description})

import pandas as pd
df = pd.DataFrame(data)

print(df.columns)
print(df.head())


Index(['title', 'description'], dtype='object')
                        title  \
0   Outdated browser detected   
1  Make Wise Talent Decisions   
2     Drive Hiring Efficiency   
3     Delight Your Candidates   
4                               

                                         description  
0        We recommend upgrading to a modern browser.  
1  Evaluate job-related behaviors, skills, and co...  
2  Our assessments provide actionable, timely ins...  
3  SHL prioritizes candidate experiences to ensur...  
4                           No description available  


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF vectorizer and fit on the descriptions
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['description'])

print("TF-IDF matrix shape:", X.shape)


TF-IDF matrix shape: (22, 54)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_assessments(query, top_n=10):
    # Vectorize the input query
    query_vec = vectorizer.transform([query])

    # Calculate cosine similarity
    similarity = cosine_similarity(query_vec, X).flatten()

    # Get indices of top N similar items
    indices = similarity.argsort()[::-1][:top_n]

    # Return the top assessments
    return df.iloc[indices][['title', 'description']]


In [ ]:
# Example test
query = "I want to assess leadership and cognitive skills"
recommendations = recommend_assessments(query)
print(recommendations)


                                                title  \
1                          Make Wise Talent Decisions   
21              Explore SHL’s Wide Range of Solutions   
19                         Are SHL assessments valid?   
18  Is it better to use psychometric assessments o...   
17  Can psychometric assessments be tailored for a...   
20                          Are SHL assessments fair?   
16         Why should I use psychometric assessments?   
15                 What are psychometric assessments?   
13                               What is the SHL OPQ?   
14                      Are SHL assessments reliable?   

                                          description  
1   Evaluate job-related behaviors, skills, and co...  
21  With our platform of pre-configured talent acq...  
19                           No description available  
18                           No description available  
17                           No description available  
20                           No desc

In [ ]:
for heading in soup.find_all(['h2', 'h3']):
    paras = []
    next_el = heading.find_next_sibling()
    while next_el and next_el.name == 'p':
        paras.append(next_el.get_text(strip=True))
        next_el = next_el.find_next_sibling()
    description = " ".join(paras) if paras else "No description available"
    data.append({
        "title": heading.get_text(strip=True),
        "description": description
    })


In [ ]:
df = df[df['description'] != "No description available"]


In [ ]:
import random

# Add dummy metadata to each row
df['url'] = "https://www.shl.com/solutions/products/product-catalog/"  # You can update later with actual links

df['remote_testing_support'] = [random.choice(['Yes', 'No']) for _ in range(len(df))]
df['adaptive_irt_support'] = [random.choice(['Yes', 'No']) for _ in range(len(df))]
df['duration_minutes'] = [random.choice([20, 30, 40, 45, 60]) for _ in range(len(df))]
df['test_type'] = [random.choice(['Cognitive', 'Technical', 'Personality', 'Behavioral']) for _ in range(len(df))]

# Preview
df.head()


,title,description,url,remote_testing_support,adaptive_irt_support,duration_minutes,test_type
0,Outdated browser detected,We recommend upgrading to a modern browser.,https://www.shl.com/solutions/products/product...,Yes,No,30,Behavioral
1,Make Wise Talent Decisions,"Evaluate job-related behaviors, skills, and co...",https://www.shl.com/solutions/products/product...,Yes,No,30,Personality
2,Drive Hiring Efficiency,"Our assessments provide actionable, timely ins...",https://www.shl.com/solutions/products/product...,Yes,No,20,Cognitive
3,Delight Your Candidates,SHL prioritizes candidate experiences to ensur...,https://www.shl.com/solutions/products/product...,No,Yes,40,Cognitive
21,Explore SHL’s Wide Range of Solutions,With our platform of pre-configured talent acq...,https://www.shl.com/solutions/products/product...,No,Yes,30,Behavioral


In [ ]:
def recommend_assessments(query, top_n=10):
    query_vec = vectorizer.transform([query])
    similarity_scores = cosine_similarity(query_vec, X).flatten()

    # Fix: Restrict top_n to number of rows in TF-IDF matrix
    top_n = min(top_n, X.shape[0])

    # Get top indices sorted by similarity
    top_indices = similarity_scores.argsort()[-top_n:][::-1]

    # Ensure indices are within df's range
    valid_indices = [i for i in top_indices if i < len(df)]

    recommendations = df.iloc[valid_indices].copy()
    recommendations['similarity_score'] = similarity_scores[valid_indices]

    return recommendations[['title', 'description', 'url', 'remote_testing_support',
                            'adaptive_irt_support', 'duration_minutes', 'test_type', 'similarity_score']]


In [ ]:
query = "Looking to hire Python developers with good communication and cognitive skills"
recommend_assessments(query)


,title,description,url,remote_testing_support,adaptive_irt_support,duration_minutes,test_type,similarity_score
1,Make Wise Talent Decisions,"Evaluate job-related behaviors, skills, and co...",https://www.shl.com/solutions/products/product...,Yes,No,30,Personality,0.367207


In [ ]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.8 MB/s eta 0:00:00


In [ ]:
import pickle

# Save the cleaned and updated dataframe with metadata
pickle.dump(df, open("df.pkl", "wb"))

# Save the TF-IDF matrix
pickle.dump(X, open("X.pkl", "wb"))

# Save the vectorizer
pickle.dump(vectorizer, open("vectorizer.pkl", "wb"))


In [ ]:
import pickle

# Check and save each variable one by one
if 'df' in locals():
    with open("df.pkl", "wb") as f:
        pickle.dump(df, f)
else:
    print("❌ 'df' not found!")

if 'X' in locals():
    with open("X.pkl", "wb") as f:
        pickle.dump(X, f)
else:
    print("❌ 'X' not found!")

if 'vectorizer' in locals():
    with open("vectorizer.pkl", "wb") as f:
        pickle.dump(vectorizer, f)
else:
    print("❌ 'vectorizer' not found!")


In [ ]:
from google.colab import files

files.download("df.pkl")
files.download("X.pkl")
files.download("vectorizer.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%%writefile app.py
import streamlit as st
import pickle
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load pre-trained files
df = pickle.load(open("df.pkl", "rb"))
X = pickle.load(open("X.pkl", "rb"))
vectorizer = pickle.load(open("vectorizer.pkl", "rb"))

# Streamlit App
st.set_page_config(page_title="SHL Assessment Recommender", layout="wide")

st.title("🔍 SHL Assessment Recommendation System")
st.markdown("Enter a job description or hiring need, and get the most relevant SHL assessments.")

# Input text box
query = st.text_area("📝 Enter Job Description / Hiring Requirement:")

if st.button("🔎 Recommend Assessments"):
    if query.strip() == "":
        st.warning("Please enter a valid job description.")
    else:
        # Vectorize the input query
        query_vec = vectorizer.transform([query])

        # Compute cosine similarity
        similarity_scores = cosine_similarity(query_vec, X).flatten()
        top_indices = similarity_scores.argsort()[-10:][::-1]  # Top 10

        # Prepare results
        recommendations = df.iloc[top_indices].copy()
        recommendations["Similarity Score"] = similarity_scores[top_indices]

        st.subheader("🎯 Top Recommended Assessments")
        for idx, row in recommendations.iterrows():
            st.markdown(f"### [{row['Assessment Name']}]({row['URL']})")
            st.write(f"**Test Type:** {row.get('Test Type', 'N/A')}")
            st.write(f"**Duration:** {row.get('Duration', 'N/A')}")
            st.write(f"**Supports Remote Testing:** {row.get('Supports Remote Testing', 'N/A')}")
            st.write(f"**Adaptive/IRT:** {row.get('Adaptive/IRT', 'N/A')}")
            st.progress(min(row['Similarity Score'], 1.0))
            st.markdown("---")


Writing app.py


In [ ]:
with open("requirements.txt", "w") as f:
    f.write("streamlit\npandas\nscikit-learn\nnumpy")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the SHL Product Catalog
url = "https://www.shl.com/en/assessments/products/"

# Send a GET request to the webpage
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Extract assessment blocks
assessment_blocks = soup.find_all("a", class_="product-list-item")

# Lists to hold data
titles, descriptions, urls = [], [], []

# Loop over blocks to extract data
for block in assessment_blocks:
    title = block.find("div", class_="product-title")
    description = block.find("div", class_="product-description")
    href = block["href"]

    titles.append(title.text.strip() if title else "Unnamed Assessment")
    descriptions.append(description.text.strip() if description else "")
    urls.append("https://www.shl.com" + href)

# Create DataFrame
df = pd.DataFrame({
    "title": titles,
    "description": descriptions,
    "url": urls,
    "remote_testing_support": ["N/A"] * len(titles),  # Placeholder
    "adaptive_irt_support": ["N/A"] * len(titles),     # Placeholder
    "duration_minutes": ["N/A"] * len(titles),         # Placeholder
    "test_type": ["N/A"] * len(titles)                 # Placeholder
})

# Save to CSV
df.to_csv("shl_assessments.csv", index=False)
print("✅ CSV file saved successfully!")


✅ CSV file saved successfully!


In [ ]:
from google.colab import files
files.download("shl_assessments.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.shl.com/en/assessments/product-catalog/"
headers = {
    "User-Agent": "Mozilla/5.0"
}

# Fetch the HTML content
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# Find all assessment cards
cards = soup.find_all("a", class_="assessment-card")

# Extract key details
assessments = []
for card in cards:
    name = card.find("h3").text.strip()
    link = "https://www.shl.com" + card["href"]
    detail_div = card.find("div", class_="assessment-card__summary")

    # Some metadata (like duration, type, etc.) is here
    metadata = [x.text.strip() for x in detail_div.find_all("li")] if detail_div else []

    assessments.append({
        "name": name,
        "url": link,
        "metadata": metadata
    })

# Show some results
for a in assessments[:5]:
    print("Name:", a["name"])
    print("URL:", a["url"])
    print("Metadata:", a["metadata"])
    print("------")


In [ ]:
!wget -q https://storage.googleapis.com/chrome-for-testing-public/135.0.7049.52/linux64/chromedriver-linux64.zip
!unzip -o chromedriver-linux64.zip
!mv chromedriver-linux64/chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver


Archive:  chromedriver-linux64.zip
  inflating: chromedriver-linux64/LICENSE.chromedriver  
  inflating: chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver  
  inflating: chromedriver-linux64/chromedriver  


In [ ]:
!sudo apt-get remove -y google-chrome-stable
!sudo rm -f /usr/bin/chromedriver


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package google-chrome-stable


In [ ]:
!apt update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
44 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [ ]:
!find /usr -name chromedriver


/usr/lib/chromium-browser/chromedriver


In [ ]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


cp: cannot stat '/usr/lib/chromium-browser/chromedriver': No such file or directory


In [ ]:
!which chromedriver


In [ ]:
!apt update
!apt install -y chromium-browser chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin/


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,243 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,775 kB]
Fetched 4,147 kB in 2s (1,765 kB/s)
Reading package lists... Done
Building dependency tree... Done
Rea

In [ ]:
!apt-get update
!apt-get install -y chromium-driver


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
!which chromedriver


In [ ]:
!wget -O /tmp/chromedriver-linux64.zip https://storage.googleapis.com/chrome-for-testing-public/135.0.7049.52/linux64/chromedriver-linux64.zip
!unzip /tmp/chromedriver-linux64.zip -d /tmp/
!chmod +x /tmp/chromedriver-linux64/chromedriver
!mv /tmp/chromedriver-linux64/chromedriver /usr/bin/chromedriver


--2025-04-07 11:34:25--  https://storage.googleapis.com/chrome-for-testing-public/135.0.7049.52/linux64/chromedriver-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.207, 74.125.195.207, 172.253.117.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9574326 (9.1M) [application/zip]
Saving to: ‘/tmp/chromedriver-linux64.zip’

/tmp/chromedriver-l 100%[===================>]   9.13M  --.-KB/s    in 0.07s   

2025-04-07 11:34:25 (128 MB/s) - ‘/tmp/chromedriver-linux64.zip’ saved [9574326/9574326]

Archive:  /tmp/chromedriver-linux64.zip
  inflating: /tmp/chromedriver-linux64/LICENSE.chromedriver  
  inflating: /tmp/chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver  
  inflating: /tmp/chromedriver-linux64/chromedriver  


In [ ]:
!which chromedriver


/usr/bin/chromedriver


In [ ]:
!apt purge chromium-browser chromium-chromedriver -y
!rm -rf /usr/bin/chromedriver /usr/bin/chrome /usr/local/bin/chrome


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  apparmor libfuse3-3 snapd squashfs-tools systemd-hwe-hwdb udev
Use 'apt autoremove' to remove them.
The following packages will be REMOVED:
  chromium-browser* chromium-chromedriver*
0 upgraded, 0 newly installed, 2 to remove and 44 not upgraded.
After this operation, 243 kB disk space will be freed.
(Reading database ... 126689 files and directories currently installed.)
Removing chromium-chromedriver (1:85.0.4183.83-0ubuntu2.22.04.1) ...
Removing chromium-browser (1:85.0.4183.83-0ubuntu2.22.04.1) ...
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for hicolor-icon-theme (0.17-2) ...
(Reading database ... 126668 files and directories currently installed.)
Purging configuration files for chromium-browser (1:85.0.4183.83-0ubuntu2.22.04.1) ...


In [ ]:
!wget -q -O /tmp/chrome-linux64.zip https://storage.googleapis.com/chrome-for-testing-public/135.0.7049.52/linux64/chrome-linux64.zip
!unzip /tmp/chrome-linux64.zip -d /tmp/
!mv /tmp/chrome-linux64 /usr/local/bin/chrome
!ln -s /usr/local/bin/chrome/chrome /usr/bin/google-chrome


Archive:  /tmp/chrome-linux64.zip
  inflating: /tmp/chrome-linux64/ABOUT  
  inflating: /tmp/chrome-linux64/MEIPreload/manifest.json  
  inflating: /tmp/chrome-linux64/MEIPreload/preloaded_data.pb  
  inflating: /tmp/chrome-linux64/PrivacySandboxAttestationsPreloaded/manifest.json  
  inflating: /tmp/chrome-linux64/PrivacySandboxAttestationsPreloaded/privacy-sandbox-attestations.dat  
  inflating: /tmp/chrome-linux64/WidevineCdm/LICENSE  
  inflating: /tmp/chrome-linux64/WidevineCdm/_platform_specific/linux_x64/libwidevinecdm.so  
  inflating: /tmp/chrome-linux64/WidevineCdm/manifest.json  
  inflating: /tmp/chrome-linux64/chrome  
  inflating: /tmp/chrome-linux64/chrome-wrapper  
  inflating: /tmp/chrome-linux64/chrome_100_percent.pak  
  inflating: /tmp/chrome-linux64/chrome_200_percent.pak  
  inflating: /tmp/chrome-linux64/chrome_crashpad_handler  
  inflating: /tmp/chrome-linux64/chrome_sandbox  
  inflating: /tmp/chrome-linux64/deb.deps  
  inflating: /tmp/chrome-linux64/icudtl.d

In [ ]:
!curl -L -o /tmp/chromedriver-linux64.zip https://storage.googleapis.com/chrome-for-testing-public/135.0.7049.52/linux64/chromedriver-linux64.zip
!unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
!chmod +x /tmp/chromedriver-linux64/chromedriver
!mv /tmp/chromedriver-linux64/chromedriver /usr/bin/chromedriver


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9349k  100 9349k    0     0  15.1M      0 --:--:-- --:--:-- --:--:-- 15.2M
Archive:  /tmp/chromedriver-linux64.zip
  inflating: /tmp/chromedriver-linux64/LICENSE.chromedriver  
  inflating: /tmp/chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver  
  inflating: /tmp/chromedriver-linux64/chromedriver  


In [ ]:
!chmod +x /tmp/chromedriver-linux64/chromedriver
!mv /tmp/chromedriver-linux64/chromedriver /usr/bin/chromedriver
!which chromedriver
!chromedriver --version


chmod: cannot access '/tmp/chromedriver-linux64/chromedriver': No such file or directory
mv: cannot stat '/tmp/chromedriver-linux64/chromedriver': No such file or directory
/usr/bin/chromedriver
ChromeDriver 135.0.7049.52 (9ba7e609d28c509a8ce9265c2247065d8d251173-refs/branch-heads/7049_41@{#4})


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

service = Service("/usr/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.google.com")
print(driver.title)  # Should print: Google
driver.quit()


Google


In [ ]:
df["title"] = df["title"].fillna("").astype(str)
df["description"] = df["description"].fillna("").astype(str)
df["combined"] = df["title"] + " " + df["description"]

print("Sample combined entries:")
print(df["combined"].head())


Sample combined entries:
Series([], Name: combined, dtype: object)


In [ ]:
df = pd.DataFrame({
    "title": ["Cognitive Ability", "Personality Test"],
    "description": ["Measures logic and reasoning", "Assesses behavioral traits"],
    "url": ["https://example.com/1", "https://example.com/2"],
    "remote_testing_support": ["Yes", "Yes"],
    "adaptive_irt_support": ["No", "Yes"],
    "duration_minutes": [30, 20],
    "test_type": ["Cognitive", "Personality"]
})
df["combined"] = df["title"] + " " + df["description"]


In [2]:
import pandas as pd

# Load the CSV
df = pd.read_csv("shl_sample_100_assessments.csv")

# Show the first few rows
df.head()


,title,description,url,remote_testing_support,adaptive_irt_support,duration_minutes,test_type
0,Assessment 1,This is a simulated description for Assessment...,https://www.shl.com/assessment-1,No,No,15,Technical
1,Assessment 2,This is a simulated description for Assessment...,https://www.shl.com/assessment-2,Yes,Yes,50,Behavioral
2,Assessment 3,This is a simulated description for Assessment...,https://www.shl.com/assessment-3,Yes,Yes,60,Behavioral
3,Assessment 4,This is a simulated description for Assessment...,https://www.shl.com/assessment-4,No,No,50,Situational Judgment
4,Assessment 5,This is a simulated description for Assessment...,https://www.shl.com/assessment-5,Yes,No,10,Situational Judgment


In [3]:
# Combine text for vectorization
df['combined'] = df['title'] + " " + df['description'] + " " + df['test_type']


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['combined'])


In [5]:
import pickle

# Save df
with open("df.pkl", "wb") as f:
    pickle.dump(df, f)

# Save TF-IDF matrix
with open("X.pkl", "wb") as f:
    pickle.dump(X, f)

# Save vectorizer
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)


In [7]:
from google.colab import files

files.download("df.pkl")
files.download("X.pkl")
files.download("vectorizer.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>